In [1]:
import metaflow
metaflow.__version__

'2.4.3'

In [2]:
import numpy as np
np.__version__

'1.19.5'

In [3]:
import config
import os

In [4]:
npy = os.path.join(config.raw_dir, config.dataset, 'kepsilon_PHLL_case_1p2_Tensors.npy')
h = np.load(npy)

In [5]:
h.shape

(14751, 10, 3, 3)

In [6]:
tensors = os.path.join(config.processed_dir, 'tensors.npy')
h = np.load(tensors)

In [7]:
h.shape

(791490, 10, 3, 3)

In [8]:
import torch

In [9]:
torch.__version__

'1.9.0'

In [10]:
import data

dataset = data.Dataset()

In [11]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [12]:
for data in dataloader:
    tensors, invariants, labels = data
    print(invariants.shape)
    break

torch.Size([128, 47])


In [ ]:
from torch import nn

dtype = torch.double

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.in_ = nn.Linear(47, 20, dtype=dtype)
        self.hidden0 = nn.Linear(20, 20, dtype=dtype)
        self.hidden1 = nn.Linear(20, 20, dtype=dtype)
        self.hidden2 = nn.Linear(20, 20, dtype=dtype)
        self.hidden3 = nn.Linear(20, 10, dtype=dtype)
        
        self.selu = nn.SELU()
    
    def forward(self, x):
        tensors, invariants = x
        out = self.in_(invariants)
        out = self.selu(out)
        out = self.hidden0(out)
        out = self.selu(out)
        out = self.hidden1(out)
        out = self.selu(out)
        out = self.hidden2(out)
        out = self.selu(out)
        out = self.hidden3(out)
        out = self.selu(out)
        out = out.reshape((-1, 10, 1, 1))
        out = out.broadcast_to((out.shape[0], 10, 3, 3))
        print(out.shape)
        print(tensors.shape)
        out = out * tensors
        print(out.shape)
        out = out.reshape((-1, 1))
        print(out.shape)
        
model = Model()

In [ ]:
for data in dataloader:
    invariants, tensors, labels = data
    model((invariants, tensors))
    break

In [13]:
from torch import nn
model = nn.Sequential(
    nn.Linear(47, 20),
    nn.Linear(20, 20),
    nn.Linear(20, 20),
    nn.Linear(20, 20),
    nn.Linear(20, 10),
)

In [ ]:

    nn.Linear(4, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 20, dtype=dtype),
    nn.Tanh(),
    nn.Linear(20, 3, dtype=dtype),

In [ ]:
epochs = 100

for epoch in epochs:
    

In [ ]:
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()

#The model has two inputs, a set of input features with a learned mapping, and the tensor basis layer
input_layer = keras.layers.Input(shape=(input_shape),name ='input_layer')
input_tensor_basis = keras.layers.Input(shape=(10,3,3),name='Tensor_input_layer')

#Hidden layer definition
hidden1 = keras.layers.Dense(20,name='Hidden1', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(hidden2)

#The layer of gn, which are coefficients for each of the ten Tn
gn = keras.layers.Dense(10,name='gn', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-4), activation = "selu")(hidden3)

#Multiply the gn by Tn, with the output being the anisotropy tensor
shaped = keras.layers.Reshape((10,1,1),name='Shape_for_dot_product')(gn)
merge = keras.layers.Dot(axes=1, name='Dot_product')([shaped,input_tensor_basis])

#Reshape the output anisotropy tensor, and trim out duplicate values (it is a symmetric matrix). The end result is a 6 component vector.
shaped_output = keras.layers.Reshape((9,1),name='Shaped_output')(merge)
trimmed_output1 = keras.layers.Lambda(lambda x : x[:,0])(shaped_output)
trimmed_output2 = keras.layers.Lambda(lambda x : x[:,1])(shaped_output)
trimmed_output3 = keras.layers.Lambda(lambda x : x[:,2])(shaped_output)
trimmed_output4 = keras.layers.Lambda(lambda x : x[:,4])(shaped_output)
trimmed_output5 = keras.layers.Lambda(lambda x : x[:,5])(shaped_output)
trimmed_output6 = keras.layers.Lambda(lambda x : x[:,8])(shaped_output)
merged_output = tf.keras.layers.Concatenate()([trimmed_output1,trimmed_output2,trimmed_output3,trimmed_output4,trimmed_output5,trimmed_output6])


model=keras.Model(inputs=[input_layer, input_tensor_basis], outputs=[merged_output])


optimizer = tf.keras.optimizers.Nadam(learning_rate = 5E-4, clipnorm=1000)
model.compile(optimizer,loss='mse',metrics=['mae', 'mse'])
print(model.summary())
